# Engenharia de Atributos

In [254]:
import pandas as pd
import matplotlib.pyplot as plt

In [255]:
merged_dataset = pd.read_csv('../data/processed/merged_dataset.csv')

C:\Users\Matheus\AppData\Local\Temp\ipykernel_2252\3999875462.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_dataset = pd.read_csv('../data/processed/merged_dataset.csv')


Convertendo as variáveis para os seus respectivos tipos corretos 

In [256]:
# Conversão dos tipos numéricos e categóricos
merged_dataset = merged_dataset.astype({
    'ID_CLIENTE': 'int64',
    'VALOR_A_PAGAR': 'float64',
    'TAXA': 'float64',
    'RENDA_MES_ANTERIOR': 'float64',
    'NO_FUNCIONARIOS': 'int64',
    'FLAG_PF': 'int', 
    'DIAS_ATRASO': 'Int64',
    'TARGET_INADIMPLENCIA': 'int',
    'SEGMENTO_INDUSTRIAL': 'category',
    'PORTE': 'category',
    'CEP_2_DIG': 'category',
    'DIAS_ADIANTAMENTO': 'int',
    'DOMINIO_EMAIL': 'category',
})

# Conversão de datas
merged_dataset['SAFRA_REF'] = pd.to_datetime(
    merged_dataset['SAFRA_REF'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(
    merged_dataset['DATA_EMISSAO_DOCUMENTO'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_PAGAMENTO'] = pd.to_datetime(
    merged_dataset['DATA_PAGAMENTO'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_VENCIMENTO'] = pd.to_datetime(
    merged_dataset['DATA_VENCIMENTO'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_CADASTRO'] = pd.to_datetime(
    merged_dataset['DATA_CADASTRO'], format='%Y-%m-%d', errors='coerce'
)


In [257]:
merged_dataset.dtypes

ID_CLIENTE                         int64
SAFRA_REF                 datetime64[ns]
DATA_EMISSAO_DOCUMENTO    datetime64[ns]
DATA_PAGAMENTO            datetime64[ns]
DATA_VENCIMENTO           datetime64[ns]
VALOR_A_PAGAR                    float64
TAXA                             float64
RENDA_MES_ANTERIOR               float64
NO_FUNCIONARIOS                    int64
DATA_CADASTRO             datetime64[ns]
FLAG_PF                            int32
SEGMENTO_INDUSTRIAL             category
DOMINIO_EMAIL                   category
PORTE                           category
CEP_2_DIG                       category
DIAS_ATRASO                        Int64
TARGET_INADIMPLENCIA               int32
DIAS_ADIANTAMENTO                  int32
dtype: object

### Criando novas features

Calculando tempo de casa. Indica a relação dos clientes com a empresa, isso pode influenciar na inadimplência. Clientes mais velhos de casa têm menos tendência a inadimplência.

In [258]:
merged_dataset['TEMPO_CADASTRO_PARA_VENCIMENTO'] = (merged_dataset['DATA_VENCIMENTO'] - merged_dataset['DATA_CADASTRO']).dt.days

In [259]:
merged_dataset['TEMPO_DE_CASA_MESES'] = merged_dataset['TEMPO_CADASTRO_PARA_VENCIMENTO'] // 30


In [260]:
merged_dataset = merged_dataset.drop(columns='TEMPO_CADASTRO_PARA_VENCIMENTO')


Prazo para o pagamento: ajuda a entender o tempo que o cliente teve para se preparar para o pagamento. Prazos curtos podem influenciar na inadimplência.



In [261]:
merged_dataset['PRAZO_PAGAMENTO_DIAS'] = (merged_dataset['DATA_VENCIMENTO'] - merged_dataset['DATA_EMISSAO_DOCUMENTO']).dt.days

Criando variável para capturar a sazonalidade. Capturar a sazonalidade em problemas de inadimplência é importante porque o comportamento de pagamento dos clientes pode variar ao longo do ano devido a fatores como 13º salário, impostos, datas comemorativas e ciclos de renda em determinados setores. Esses elementos influenciam diretamente a capacidade de pagamento e o risco de inadimplência. Além disso, incluir variáveis sazonais, como o mês de referência da transação, ajuda o modelo a reconhecer padrões temporais recorrentes, melhorando sua capacidade preditiva.

In [262]:
merged_dataset['MES_SAFRA'] = merged_dataset['SAFRA_REF'].dt.month

 Criação de **INADIMPLENCIAS\_ANTERIORES**. Essa variável é um contador cumulativo do número de inadimplências passadas para cada cliente até o momento do pagamento atual. Ela captura o comportamento passado do cliente, que é um forte indicador de risco de inadimplência. Clientes reincidentes tendem a ter maior probabilidade de inadimplir novamente. Para evitar **data leakage**, a contagem considera apenas as inadimplências anteriores à linha atual, utilizando o deslocamento dos valores (shift) antes do cálculo acumulado (cumsum). Dessa forma, garantimos que o modelo não tenha acesso a informações futuras no momento da previsão.


In [263]:
# Ordena por cliente e data de emissão
merged_dataset = merged_dataset.sort_values(by=['ID_CLIENTE', 'DATA_EMISSAO_DOCUMENTO'])

merged_dataset['INADIMPLENCIAS_ANTERIORES'] = (
    merged_dataset
    .groupby('ID_CLIENTE')['TARGET_INADIMPLENCIA']
    .shift(fill_value=0)  # desloca a inadimplência para não pegar a atual (evita data leakage)
    .groupby(merged_dataset['ID_CLIENTE'])
    .cumsum()
)

# colocando um limite para essa abordagem para não prejudicar o modelo como um todo 
merged_dataset['INADIMPLENCIAS_ANTERIORES']=merged_dataset['INADIMPLENCIAS_ANTERIORES'].clip(upper=15)

A consulta a seguir mostra que a estratégia funcionou de maneira adequada e a feature foi criada de maneira correta

In [264]:
merged_dataset[merged_dataset['ID_CLIENTE']==43266641219008038][['ID_CLIENTE', 'DATA_EMISSAO_DOCUMENTO', 'TARGET_INADIMPLENCIA', 'INADIMPLENCIAS_ANTERIORES']]


,ID_CLIENTE,DATA_EMISSAO_DOCUMENTO,TARGET_INADIMPLENCIA,INADIMPLENCIAS_ANTERIORES
309,43266641219008038,2020-02-24,1,0
310,43266641219008038,2020-02-25,1,1
311,43266641219008038,2020-03-01,1,2
312,43266641219008038,2020-03-03,1,3
313,43266641219008038,2020-03-06,1,4
314,43266641219008038,2020-03-15,1,5
315,43266641219008038,2020-03-15,1,6
316,43266641219008038,2020-03-15,1,7
317,43266641219008038,2020-07-02,1,8


Criação da coluna ADIANTAMENTOS_ANTERIORES. Usar o histórico de adiantamentos como feature pode ser útil em modelos de inadimplência, pois clientes que pagam antes do vencimento tendem a apresentar menor risco, revelando disciplina financeira e previsibilidade de caixa. Para evitar **data leakage**, a contagem considera apenas os pagamentos realizados antes da data da cobrança analisada, sem utilizar informações futuras que estariam indisponíveis no momento da previsão.


In [265]:
# criação de coluna temporarea para armazenar os adiantamentos 
merged_dataset['HOUVE_ADIANTAMENTO'] = (merged_dataset['DIAS_ADIANTAMENTO'] >= 1).astype(int)

merged_dataset['ADIANTAMENTOS_ANTERIORES'] = (
    merged_dataset
    .groupby('ID_CLIENTE')['HOUVE_ADIANTAMENTO']
    .shift(fill_value=0)  # desloca a inadimplência para não pegar a atual (evita data leakage)
    .groupby(merged_dataset['ID_CLIENTE'])
    .cumsum()
)

merged_dataset['ADIANTAMENTOS_ANTERIORES']=merged_dataset['ADIANTAMENTOS_ANTERIORES'].clip(upper=15)

merged_dataset = merged_dataset.drop(columns='HOUVE_ADIANTAMENTO')

Conferindo a criação da feature. Feature funcionou de maneira adequada

In [266]:
merged_dataset[merged_dataset['ID_CLIENTE']==8784237149961904][['ID_CLIENTE', 'DATA_EMISSAO_DOCUMENTO', 'TARGET_INADIMPLENCIA','DIAS_ADIANTAMENTO',  'ADIANTAMENTOS_ANTERIORES']]

,ID_CLIENTE,DATA_EMISSAO_DOCUMENTO,TARGET_INADIMPLENCIA,DIAS_ADIANTAMENTO,ADIANTAMENTOS_ANTERIORES
0,8784237149961904,2018-09-04,0,0,0
1,8784237149961904,2018-09-06,0,0,0
2,8784237149961904,2018-09-09,0,0,0
3,8784237149961904,2018-09-11,0,0,0
4,8784237149961904,2018-09-17,0,0,0
...,...,...,...,...,...
231,8784237149961904,2021-06-10,0,0,5
232,8784237149961904,2021-06-11,0,0,5
233,8784237149961904,2021-06-12,0,0,5
234,8784237149961904,2021-06-14,0,0,5


In [267]:
merged_dataset['ADIANTAMENTOS_ANTERIORES']

merged_dataset.sort_values(by='ADIANTAMENTOS_ANTERIORES', ascending=False)


,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,DATA_CADASTRO,...,PORTE,CEP_2_DIG,DIAS_ATRASO,TARGET_INADIMPLENCIA,DIAS_ADIANTAMENTO,TEMPO_DE_CASA_MESES,PRAZO_PAGAMENTO_DIAS,MES_SAFRA,INADIMPLENCIAS_ANTERIORES,ADIANTAMENTOS_ANTERIORES
53591,6964108750473070287,2019-10-01,2019-10-06,2019-11-08,2019-11-11,101352.00,5.99,537035.0,108,2000-08-15,...,MEDIO,42,0,0,3,234,36,10,1,15
25732,3355881107559250653,2020-09-01,2020-09-25,2020-10-13,2020-10-13,12980.00,11.99,121896.0,109,2011-02-14,...,GRANDE,32,0,0,0,117,18,9,1,15
25718,3355881107559250653,2020-08-01,2020-08-25,2020-09-10,2020-09-11,41474.00,6.99,123607.0,118,2011-02-14,...,GRANDE,32,0,0,1,116,17,8,1,15
25719,3355881107559250653,2020-08-01,2020-08-26,2020-09-14,2020-09-14,25990.00,6.99,123607.0,118,2011-02-14,...,GRANDE,32,0,0,0,116,19,8,1,15
25720,3355881107559250653,2020-08-01,2020-08-28,2020-09-14,2020-09-14,20753.00,6.99,123607.0,118,2011-02-14,...,GRANDE,32,0,0,0,116,17,8,1,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34470,4661684997487171886,2019-03-01,2019-03-27,2019-04-15,2019-04-12,15342.15,6.99,123814.0,144,2015-03-02,...,GRANDE,29,3,0,0,50,16,3,1,0
34471,4661684997487171886,2019-03-01,2019-03-27,2019-04-15,2019-04-12,20799.55,6.99,123814.0,144,2015-03-02,...,GRANDE,29,3,0,0,50,16,3,1,0
34472,4661684997487171886,2019-04-01,2019-04-22,2019-05-10,2019-05-08,42417.20,11.99,612040.0,145,2015-03-02,...,GRANDE,29,2,0,0,50,16,4,1,0
34473,4661684997487171886,2019-04-01,2019-04-23,2019-05-10,2019-05-09,32295.35,11.99,612040.0,145,2015-03-02,...,GRANDE,29,1,0,0,50,16,4,1,0


In [268]:
merged_dataset['INADIMPLENCIAS_ANTERIORES']

0        0
1        0
2        0
3        0
4        0
        ..
72485    2
72486    2
72487    2
72488    2
72489    2
Name: INADIMPLENCIAS_ANTERIORES, Length: 72490, dtype: int32

Selecionando uma amostra para conferir se o que eu fiz funcionou corretamente 

In [269]:
merged_dataset[merged_dataset['ID_CLIENTE']==8784237149961904][['DATA_EMISSAO_DOCUMENTO','DATA_VENCIMENTO','DATA_PAGAMENTO','DIAS_ADIANTAMENTO', 'ADIANTAMENTOS_ANTERIORES', 'INADIMPLENCIAS_ANTERIORES', 'TARGET_INADIMPLENCIA']]

,DATA_EMISSAO_DOCUMENTO,DATA_VENCIMENTO,DATA_PAGAMENTO,DIAS_ADIANTAMENTO,ADIANTAMENTOS_ANTERIORES,INADIMPLENCIAS_ANTERIORES,TARGET_INADIMPLENCIA
0,2018-09-04,2018-09-24,2018-09-24,0,0,0,0
1,2018-09-06,2018-09-24,2018-09-24,0,0,0,0
2,2018-09-09,2018-09-25,2018-09-25,0,0,0,0
3,2018-09-11,2018-09-27,2018-09-27,0,0,0,0
4,2018-09-17,2018-10-03,2018-10-03,0,0,0,0
...,...,...,...,...,...,...,...
231,2021-06-10,2021-06-28,2021-06-28,0,5,1,0
232,2021-06-11,2021-07-01,2021-07-01,0,5,1,0
233,2021-06-12,2021-06-29,2021-06-29,0,5,1,0
234,2021-06-14,2021-06-30,2021-06-30,0,5,1,0


Analisando a correlação de todas as features com a variável target

In [270]:
#merged_dataset.to_csv('../data/processed/dataset_features_v1.csv', index=False)

# LIDAR COM A ALTA CARDINALIDADE DE CEP_2_DIG POSTERIORMENTE
